# 2.2 Pandas, Basic Mapping

This section aims to provide new skills in python to handle structured "table" data. 

Learning outcome:
-   Manipulation of data frames (describing, filtering, ...) 
-   Learn about Lambda functions
-   Intro to datetime objects
-   Plotting data from data frames (histograms and maps)
-   Introduction to Plotly
-   Introduction to CSV & Parquet


We will work on several structured data sets: sensor metadata, seismic data product (earthquake catalog).

First, we import all the modules we need:

In [ ]:
import numpy as np
import pandas as pd
import io
import requests
import time
from datetime import datetime, timedelta

import plotly.express as px
import plotly.io as pio
# pio.renderers.default = 'vscode' # writes as standalone html, 
# pio.renderers.default = 'iframe' # writes files as standalone html, 
# pio.renderers.default = 'png' # writes files as standalone pnl, 
# try notebook, jupyterlab, png, vscode, iframe


# The Basics of Pandas 

Pandas are composed of ``Series`` and ``DataFrame``. ``Series`` are columns with attributes or keys. The ``DataFrame`` is a multi-dimensional table made up of ``Series``.

We can create a DataFrame composed of series from scratch using Python dictionary:

In [ ]:
data = {
    'temperature' : [36,37,30,50],
    'precipitation':[3,1,0,0]
}
my_pd = pd.DataFrame(data)
print(my_pd)

Each (key,value) item in the dataframe correspond to a value in ``data``. To get the keys of the dataframe, type:

In [ ]:
my_pd.keys()

get a specific ``Series`` (different from the array)

In [ ]:
print(my_pd.temperature[:])
print(type(my_pd.temperature[:]))

to get the _value_ of a specific key (e.g., temperature), at a specific index (e.g., 2) type:

In [ ]:
print(my_pd.temperature[2])
print(type(my_pd.temperature[2]))

# Reading a DataFrame from a CSV file

We can read a pandas directly from a standard file. Here you will read a catatalog of earthquakes.

In [ ]:
quake = pd.read_csv("Global_Quakes_IRIS.csv")

Now you use the ``head`` function to display what is in the file

In [ ]:
# enter answer here
quake.head()

Display the depth using two ways to use the pandas object

In [ ]:
print(quake.depth)
print(quake['depth'])

Calculate basic statitics of the data using the function ``describe``.

In [ ]:
quake.describe()

Calculate mean and median of specific ``Series``, for example depth.

In [ ]:
# answer it here
print(quake.depth.mean())
print(quake.depth.median())


## Simple Python Functions
We will now practice how to modify the content of the DataFrame using functions. We will take the example that we want to change the depth values from meters to kilometers. First we can define this operation as a function

In [ ]:
# this function converts a value in meters to a value in kilometers
m2km = 1000 # this is defined as a global variable
def meters2kilometers(x):
    return x/m2km


In [ ]:
# now test it using the first element of the quake DataFrame
meters2kilometers(quake.depth[0])

Let's define another function that uses a local instead of global variable

In [ ]:
def meters2kilometers2(x):
    m2km2=1000
    return x/m2km2
# m2km2 is a local variable and cannot be called outside of the function. Prove it next by inquiring its value in the next cell.

In [ ]:
print(m2km2)

We now discuss the **lambda** functions.

In [ ]:
# now we apply it on the entire Series
meters2kilometers(quake.depth)

We can also define this very basic function as a **lambda** function. There are several ways of doing an operation on all rows of a column. The first option is to use the map function.

If you are not familiar with lambda function in Python, look at:

https://realpython.com/python-lambda/

We will practice a bit lambda functions



In [ ]:
# Now the equivalent in lambda is:
lambda_meters2kilometers = lambda x:x/1000
# x is the variable

In [ ]:
# apply it to the entire series
lambda_meters2kilometers(quake.depth)

In [ ]:
# you can add several variables into lambda functions
remove_anything = lambda x,y:x-y
remove_anything(3,2)

This did not affect the values of the DataFrame, check it:

In [ ]:
quake.depth

Instead, you could overwrite ``quake.depth=X``. Try two approaches but just do it once!

In [ ]:
#type answer below
quake.depth=quake.depth.map(lambda x:x/1000)

In [ ]:
# or like this
# quake.depth=quake.depth.apply(lambda x:x/1000)

Plot a histogram of the depth distributions using matplotlib function ``hist``.

In [ ]:
# answer here
plt.hist(quake.depth,100)
plt.grid(True)
plt.xlabel('Quake depth (km)')
plt.show()

You can use the interactive plotting package Plotly. First we will show a histogram of the event depth using the function ``histogram``

In [ ]:
fig = px.histogram(quake,   #specify what dataframe to use
             x="depth",  #specify the variable for the histogram 
             nbins=50,       #number of bins for the histogram 
             height=400,     #dimensions of the figure
             width=600);
fig.show()

We will now make a new plot of the location of the earthquakes. We will use Plotly tool. 

The markersize will be scaled with the earthquake magnitude. To do so, we add a ``marker_size`` series in the DataFrame

In [ ]:
quake['marker_size'] =np.fix(np.exp(quake['magnitude'])) # add marker size as exp(mag)
quake['magnitude bin'] = 0.5*np.fix(2*quake['magnitude']) # add marker size as exp(mag)

## Mapping using Plotly

Now we will plot the earthquakes locations on a map using the Plotly package. More tutorials on [Plotly](https://plotly.com/). Input of the function in the function is self-explanatory and typical of Python's function. The code [documentation](https://plotly.com/python/scatter-plots-on-maps/) of Plotly scatter_geo lists the variables.

In [ ]:
fig = px.scatter_geo(quake,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),
                     height=600, width=600,
                     size='marker_size', color='magnitude',
                     hover_name="description",
                     hover_data=['description','magnitude','depth']);
fig.update_geos(resolution=110, showcountries=True)
fig.update_geos(resolution=110, showcountries=True,projection_type="orthographic")


The data was sorted by time. We now want to sort and show the data instead by magnitude. We use the pandas function ``sort`` to create a new DataFrame with sorted values.

In [ ]:
quakes2plot=quake.sort_values(by='magnitude bin')

quakes2plot.head()

Now we will plot again using Plotly

In [ ]:
fig = px.scatter_geo(quakes2plot,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),
                     height=600, width=600,
                     size='marker_size', color='magnitude',
                     hover_name="description",
                     hover_data=['description','magnitude','depth']);
fig.update_geos(resolution=110, showcountries=True)
# fig.update_geos(resolution=110, showcountries=True,projection_type="orthographic")


## Create a Pandas from a text file.

The python package pandas is very useful to read csv files, but also many text files that are more or less formated as one observation per row and one column for each feature.

As an example, we are going to look at the list of seismic stations from the Northern California seismic network, available [here](http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day):



In [1]:
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

In [ ]:
# this gets the file linked in the URL page and convert it to a string
s = requests.get(url).content 

In [3]:

# this will convert the string, decode it , and make it a table
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
# because columns/keys were not assigned, assign them now
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

Let us look at the data. They are now stored into a pandas dataframe.

In [4]:
data.head()

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
0,AAR,NC,EHZ,--,100.0,"1984/01/01,00:00:00","1987/05/01,00:00:00",39.27594,-121.02696,911.0,0.0,-90.0,0.0
1,AAR,NC,EHZ,--,100.0,"1987/05/01,00:00:00","2006/01/04,19:19:00",39.27594,-121.02696,911.0,0.0,-90.0,0.0
2,AAR,NC,SHZ,--,20.0,"1994/11/28,00:00:00","2006/01/04,19:19:00",39.27594,-121.02696,911.0,0.0,-90.0,0.0
3,AAS,NC,EHZ,--,100.0,"1984/11/27,18:45:00","1987/05/01,00:00:00",38.43014,-121.10959,31.0,0.0,-90.0,0.0
4,AAS,NC,EHZ,--,100.0,"1987/05/01,00:00:00","3000/01/01,00:00:00",38.43014,-121.10959,31.0,0.0,-90.0,0.0


We can output the first element of the DataFrame:

In [ ]:
data.iloc[0]

In [ ]:
data.iloc[:, 0]

The DataFrame may have bad values. A typical data cleaning involves removing Nan and Zeros for instance.

Use Plotly to map the stations.

In [ ]:
data.dropna(inplace=True)
data=data[data.longitude!=0]

In [ ]:
fig = px.scatter_geo(data,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),
                     height=600, width=600,
                     hover_name="station",
                     hover_data=['network','station','channel','rate']);
fig.update_geos(resolution=110, showcountries=True)


In [ ]:
fig = px.scatter_mapbox(data,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),mapbox_style="carto-positron",
                     height=600, width=500,
                     hover_name="station",
                     hover_data=['network','station','channel','rate']);
fig.update_layout(title="Northern California Seismic Network")
fig.show()

## Pandas: data selection
We can filter the data with the value taken by a given column:

In [ ]:
data.loc[data.station=='KCPB']

In [10]:
# Select two stations, use the typical "OR" |
data.loc[(data.station=='KCPB') | (data.station=='KHBB')]

'AAR'

In [ ]:
# Select two stations, use the typical "AND" &
data.loc[(data.station=='KCPB') & (data.channel=='HNZ')]

In [ ]:
# or like this
data.loc[data.station.isin(['KCPB', 'KHBB'])]

We can filter the data with the value taken by a given column:

In [11]:
data.loc[data.station=='KCPB']

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
2993,KCPB,NC,BHE,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
2994,KCPB,NC,BHE,--,50.0,"2000/06/06,16:00:00","2002/01/24,23:50:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
2995,KCPB,NC,BHE,--,50.0,"2002/01/24,23:50:00","2002/10/16,23:59:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
2996,KCPB,NC,BHE,--,20.0,"2002/10/17,00:00:00","2006/01/24,18:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
2997,KCPB,NC,BHN,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,KCPB,NC,MNE,--,10.0,"2000/06/06,16:00:00","2000/07/12,00:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3067,KCPB,NC,MNN,--,10.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
3068,KCPB,NC,MNN,--,10.0,"2000/06/06,16:00:00","2000/07/12,00:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
3069,KCPB,NC,MNZ,--,10.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,-90.0,0.0


We can access to a brief summary of the data:

In [14]:
data.station.describe()

count     6140
unique     893
top       KCPB
freq        78
Name: station, dtype: object

In [15]:
data.elevation.describe()

count    6140.000000
mean      650.942248
std       681.708404
min     -1388.000000
25%       146.750000
50%       406.000000
75%       918.000000
max      3680.000000
Name: elevation, dtype: float64

We can perform standard operations on the whole data set:

In [16]:
data.mean()

rate          92.544982
latitude      37.930985
longitude   -121.440088
elevation    650.942248
depth         17.581270
dip          -45.894137
azimuth       18.549186
dtype: float64

In the case of a categorical variable, we can get the list of possile values that this variable can take:

In [17]:
data.channel.unique()

array(['EHZ', 'SHZ', 'HHE', 'HHN', 'HHZ', 'HNE', 'HNN', 'HNZ', 'LHE',
       'LHN', 'LHZ', 'ELE', 'ELN', 'ELZ', 'SLE', 'SLN', 'SLZ', 'LCE',
       'LCL', 'LCQ', 'LOG', 'OCF', 'VCO', 'VEA', 'VEC', 'VEP', 'VFP',
       'VKI', 'GAN', 'GNS', 'GPL', 'GST', 'VDT', 'VEI', 'VPB', 'EHE',
       'EHN', 'BNE', 'BNN', 'BN1', 'BN2', 'BN3', 'BV1', 'EP1', 'EP2',
       'EP3', 'HDO', 'HN1', 'HN2', 'HN3', 'HV1', 'SP2', 'SP3', 'BNZ',
       'LDO', 'HJ2', 'HJ3', 'HJZ', 'ACE', 'GEL', 'GLA', 'GLO', 'ATT',
       'SHE', 'SHN', 'BHE', 'BHN', 'BHZ', 'LNE', 'LNN', 'LNZ', 'MHE',
       'MHN', 'MHZ', 'MNE', 'MNN', 'MNZ', 'HH2', 'HH3', 'LH2', 'LH3',
       'SP1', 'DP1', 'DP2', 'DP3', 'HLE', 'HLN', 'HLZ', 'XNE', 'XNN',
       'XNZ', 'EH1'], dtype=object)

and get the number of times that each value is taken:

In [18]:
data.station.value_counts()

KCPB     78
KMPB     72
JSGB     69
KHMB     63
CCH1     60
         ..
PL21      1
LSS       1
IRG1A     1
AGI       1
BVLB      1
Name: station, Length: 893, dtype: int64

The second option is to use the apply function:

In [19]:
data_elevation_mean=data.elevation.unique().mean()
def remean_elevation(row):
    row.elevation = row.elevation - data_elevation_mean
    return row
data.apply(remean_elevation, axis='columns')

0       260.057752
1       260.057752
2       260.057752
3      -619.942248
4      -619.942248
           ...    
6135    427.057752
6136    427.057752
6137    427.057752
6138    427.057752
6139   -649.942248
Name: elevation, Length: 6140, dtype: float64

We can also carry out simple operations on columns, provided they make sense.

In [21]:
data.network + ' - ' + data.station

0        NC - AAR
1        NC - AAR
2        NC - AAR
3        NC - AAS
4        NC - AAS
          ...    
6135     NC - WMP
6136     NC - WMP
6137     NC - WMP
6138     NC - WMP
6139    NC - WWVB
Length: 6140, dtype: object

A useful feature is to group the rows depending on the value of a categorical variable, and then apply the same operation to all the groups. For instance, I want to know how many times each station appears in the file:

In [22]:
data.groupby('station').station.count()

station
AAR      3
AAS      3
ABJ      4
ABR      3
ADW      3
        ..
VRC      4
VSP      2
VWB      1
WMP     10
WWVB     1
Name: station, Length: 893, dtype: int64

We can have access to the data type of each column:

In [25]:
data.dtypes

station        object
network        object
channel        object
location       object
rate          float64
start_time     object
end_time       object
latitude      float64
longitude     float64
elevation     float64
depth         float64
dip           float64
azimuth       float64
dtype: object

Here, pandas does not recognize the start_time and end_time columns as a datetime format, so we cannot use datetime operations on them. We first need to convert these columns into a datetime format:

In [ ]:
data.start_time.values()

In [ ]:
type(data['start_time'][0])

In [26]:
# Transform column from string into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
print(data['start_time'] )
type(data['start_time'][0])

In [ ]:
print(data['end_time'])

In [27]:
# do the same for end times
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

station
AAR    1984-01-01 00:00:00
AAS    1984-11-27 18:45:00
ABJ    1984-01-01 00:00:00
ABR    1984-01-01 00:00:00
ADW    1984-01-01 00:00:00
               ...        
VRC    1993-09-23 22:20:00
VSP    1993-09-24 22:05:00
VWB    1985-01-01 00:00:00
WMP    1995-03-02 19:00:00
WWVB   1984-01-01 00:00:00
Length: 893, dtype: datetime64[ns]

We can now look when each seismic station was installed:

In [ ]:
data.groupby('station').apply(lambda df: df.start_time.min())

The ``agg`` function allows to carry out several operations to each group of rows:

In [28]:
data.groupby(['station']).elevation.agg(['min', 'max'])

,min,max
station,,
AAR,911.0,911.0
AAS,31.0,31.0
ABJ,434.0,434.0
ABR,-1.0,-1.0
ADW,228.0,228.0
...,...,...
VRC,1666.0,1666.0
VSP,1545.0,1545.0
VWB,1736.0,1736.0


Select the stations that were deployed first and recovered last

In [29]:
data.groupby(['station']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

,start_time,end_time
station,,
AAR,1984-01-01 00:00:00,2006-01-04 19:19:00
AAS,1984-11-27 18:45:00,2025-01-01 00:00:00
ABJ,1984-01-01 00:00:00,2019-06-26 19:17:00
ABR,1984-01-01 00:00:00,1997-08-04 21:02:00
ADW,1984-01-01 00:00:00,2006-04-20 01:08:00
...,...,...
VRC,1993-09-23 22:20:00,2001-07-12 16:50:00
VSP,1993-09-24 22:05:00,2001-07-12 16:50:00
VWB,1985-01-01 00:00:00,1985-03-25 19:00:00


We can also make groups by selecting the values of two categorical variables:

In [30]:
data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

start_time            end_time
station channel                                        
AAR     EHZ     1984-01-01 00:00:00 2006-01-04 19:19:00
        SHZ     1994-11-28 00:00:00 2006-01-04 19:19:00
AAS     EHZ     1984-11-27 18:45:00 2025-01-01 00:00:00
        SHZ     1994-11-28 00:00:00 2006-01-24 18:00:00
ABJ     EHZ     1984-01-01 00:00:00 2019-06-26 19:17:00
...                             ...                 ...
WMP     EHZ     1995-03-02 19:00:00 2002-05-08 22:30:00
        SHE     1995-07-02 12:00:00 2002-05-08 22:30:00
        SHN     1995-07-02 12:00:00 2002-05-08 22:30:00
        SHZ     1995-03-02 19:00:00 2002-05-08 22:30:00
WWVB    ATT     1984-01-01 00:00:00 1999-03-20 23:59:00

[3725 rows x 2 columns]

Previously, we just printed the output, but we can also store it in a new variable:

In [31]:
data_grouped = data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

In [32]:
data_grouped.head()

start_time            end_time
station channel                                        
AAR     EHZ     1984-01-01 00:00:00 2006-01-04 19:19:00
        SHZ     1994-11-28 00:00:00 2006-01-04 19:19:00
AAS     EHZ     1984-11-27 18:45:00 2025-01-01 00:00:00
        SHZ     1994-11-28 00:00:00 2006-01-24 18:00:00
ABJ     EHZ     1984-01-01 00:00:00 2019-06-26 19:17:00

When we select only some rows, the index is not automatically reset to start at 0. We can do it manually. Many functions in pandas have also an option to reset the index, and option to transform the dataframe in place, instead of saving the results in another variable.

In [33]:
data_grouped.reset_index()

,station,channel,start_time,end_time
0,AAR,EHZ,1984-01-01 00:00:00,2006-01-04 19:19:00
1,AAR,SHZ,1994-11-28 00:00:00,2006-01-04 19:19:00
2,AAS,EHZ,1984-11-27 18:45:00,2025-01-01 00:00:00
3,AAS,SHZ,1994-11-28 00:00:00,2006-01-24 18:00:00
4,ABJ,EHZ,1984-01-01 00:00:00,2019-06-26 19:17:00
...,...,...,...,...
3720,WMP,EHZ,1995-03-02 19:00:00,2002-05-08 22:30:00
3721,WMP,SHE,1995-07-02 12:00:00,2002-05-08 22:30:00
3722,WMP,SHN,1995-07-02 12:00:00,2002-05-08 22:30:00
3723,WMP,SHZ,1995-03-02 19:00:00,2002-05-08 22:30:00


It is also possible to sort the dataset by value.

In [34]:
data_grouped.sort_values(by='start_time')

start_time            end_time
station channel                                        
PG1     HV1     1983-06-27 00:00:00 1993-11-10 18:00:00
AAR     EHZ     1984-01-01 00:00:00 2006-01-04 19:19:00
HQR     EHZ     1984-01-01 00:00:00 1992-10-19 18:45:00
        ELE     1984-01-01 00:00:00 1992-10-19 18:45:00
        ELN     1984-01-01 00:00:00 1992-10-19 18:45:00
...                             ...                 ...
PBIB    HNE     2020-11-19 00:00:00 2025-01-01 00:00:00
PPBB    HNE     2020-11-19 00:00:00 2025-01-01 00:00:00
KBN     HNE     2021-02-26 00:48:00 2025-01-01 00:00:00
        HNN     2021-02-26 00:48:00 2025-01-01 00:00:00
        HNZ     2021-02-26 00:48:00 2025-01-01 00:00:00

[3725 rows x 2 columns]

We can apply the sorting to several columns:

In [35]:
data_grouped.sort_values(by=['start_time', 'end_time'])

start_time            end_time
station channel                                        
PG1     HV1     1983-06-27 00:00:00 1993-11-10 18:00:00
PGC     EHZ     1984-01-01 00:00:00 1984-04-12 18:00:00
AFO     EHZ     1984-01-01 00:00:00 1984-04-23 00:00:00
MCH     EHZ     1984-01-01 00:00:00 1984-08-01 19:31:00
MCN     EHZ     1984-01-01 00:00:00 1984-09-22 16:45:00
...                             ...                 ...
PPBB    VEI     2020-11-19 00:00:00 2025-01-01 00:00:00
        VPB     2020-11-19 00:00:00 2025-01-01 00:00:00
KBN     HNE     2021-02-26 00:48:00 2025-01-01 00:00:00
        HNN     2021-02-26 00:48:00 2025-01-01 00:00:00
        HNZ     2021-02-26 00:48:00 2025-01-01 00:00:00

[3725 rows x 2 columns]

## CSV vs Parquet

Parquet is a compressed data format that stores and compress the culumns. It is fast for I/O and compact formt.

Save ``data`` into a CSV file:

In [ ]:
%timeit data.to_csv("my_metadata.csv")
!ls -lh my_metadata.csv

Try and save in Parquet, compare time and memory

In [ ]:
%timeit data.to_parquet("my_metadata.pq")
!ls -lh my_metadata.pq